In [19]:
import pandas as pd
import numpy as np
import json
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import dateutil.parser as parser
from datetime import datetime, date, timedelta
import torch
import skorch
import scipy
import torch.nn as nn
import torch
import torch.nn.functional as F
import sys
from skorch.helper import DataFrameTransformer
import time
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, mean_squared_error, r2_score
from sklearn.linear_model import SGDClassifier
from skorch import NeuralNetRegressor
from sklearn.pipeline import Pipeline

In [2]:
merged = pd.read_csv('merged.csv')

all_tweets = merged["text"]
all_tweets = all_tweets.to_list()
for tweet in all_tweets:
    # Remove all the special characters
    processed_tweet = re.sub(r'\W', ' ', tweet)
 
    # remove all single characters
    processed_tweet = re.sub(r'\s+[a-zA-Z]\s+', ' ', processed_tweet)
 
    # Remove single characters from the start
    processed_tweet = re.sub(r'\^[a-zA-Z]\s+', ' ', processed_tweet) 
 
    # Substituting multiple spaces with single space
    processed_tweet= re.sub(r'\s+', ' ', processed_tweet, flags=re.I)
 
    # Removing prefixed 'b'
    processed_tweet = re.sub(r'^b\s+', '', processed_tweet)
    
    #removing common pronouns and prepositions
    processed_tweet = re.sub(r'of|to|https|keep|128', '', processed_tweet)
 
    # Converting to Lowercase
    processed_tweet = processed_tweet.lower()
    
    tweet = processed_tweet
    
tfidfv = TfidfVectorizer(max_features=2000)
df2 = tfidfv.fit_transform(all_tweets)
df2array = df2.toarray()

In [11]:
y = merged['Change']
print(y.max())
print(y.min())
print(y.mean())
bins = np.arange(start=-4.5, stop=4.5, step = 0.5)
print(bins)
ybin = np.digitize(y, bins)
print(ybin)

4.04
-4.57
-0.12666603819921832
[-4.5 -4.  -3.5 -3.  -2.5 -2.  -1.5 -1.  -0.5  0.   0.5  1.   1.5  2.
  2.5  3.   3.5  4. ]
[14 14 14 ... 11 11 11]


In [13]:
#Selecting the data and splitting into train and test
y = merged['Change']
print(type(y))
# X = df3
X = df2
# X = df2array
from sklearn.model_selection import train_test_split  
x_train, x_test, y_train, y_test = train_test_split(X, ybin, test_size=0.2, random_state=0)

# train_holdout_data.to_csv('train_holdout_data.csv', index=False)
# x_holdout.to_csv('x_holdout.csv', index=False)
# y_holdout.to_csv('y_holdout.csv', index=False)
# train_data.to_csv('train_data.csv', index=False)
# x_train.to_csv('x_train.csv', index=False)
# y_train.to_csv('y_train.csv', index=False)
# # x_test.to_csv('x_test.csv', index=False)
# y_test.to_csv('y_test.csv', index=False)

# # train_holdout_data = pd.read_csv('train_holdout_data.csv')
# # x_holdout = pd.read_csv('x_holdout.csv')
# # y_holdout = pd.read_csv('y_holdout.csv')
# # x_train = pd.read_csv('x_train.csv')
# y_train = pd.read_csv('y_train.csv')
# # x_test = pd.read_csv('x_test.csv')
# y_test = pd.read_csv('y_test.csv')

<class 'pandas.core.series.Series'>


In [28]:
print(x_train)
print(y_train)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

  (0, 189)	0.08380788459917278
  (0, 192)	0.29065709086724617
  (0, 412)	0.0910575335714549
  (0, 863)	0.09039541729721204
  (0, 882)	0.9444743978103098
  (1, 473)	0.5813504722460844
  (1, 693)	0.5688499012524305
  (1, 1267)	0.5677151152139508
  (1, 1471)	0.1270471023753213
  (2, 171)	0.25560451661590167
  (2, 189)	0.0494288988136831
  (2, 404)	0.21857836540117054
  (2, 412)	0.10740931201529029
  (2, 746)	0.16197758446860766
  (2, 754)	0.16488819985182684
  (2, 784)	0.25568969046894
  (2, 837)	0.2556257923706774
  (2, 863)	0.10662829532505955
  (2, 898)	0.16688571337199354
  (2, 899)	0.15884379540544347
  (2, 900)	0.21600695991509916
  (2, 901)	0.1873757455083385
  (2, 903)	0.16610684061773368
  (2, 927)	0.2684319943127905
  (2, 935)	0.14575692329065057
  :	:
  (413703, 171)	0.26700152344386174
  (413703, 189)	0.051632856336564566
  (413703, 404)	0.22832443388189674
  (413703, 412)	0.05609926287655328
  (413703, 576)	0.2006282645214646
  (413703, 714)	0.11487716615698566
  (413703, 746

In [29]:
pipe = Pipeline([("tfidf_vector_com", TfidfVectorizer(
               input="array",
               norm="l2",
               max_features=None,
               sublinear_tf=True,
               stop_words="english")),
          ("clf", SGDClassifier(
               loss="log",
               penalty="l2",
               class_weight="balanced",
               tol=0.001))])

pipe.fit(x_train, y_train)

pred_test = pipe.predict(x_test)
pred_train = pipe.predict(x_train)
print("test accuracy", str(np.mean(pred_test == y_test)))
print(metrics.classification_report(y_test, pred_test))

AttributeError: lower not found